In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / "_scripts"))

In [3]:
from case_edits.ezcase_new import *
from new_subsurfaces.subsurfaces import *
from geometry.wall_normal import WallNormal
from new_subsurfaces.interfaces import SubsurfaceType

In [4]:

inputs_dir = "case_red_d3"
outputs_dir = "tests/test22_svg2plan"

idf = create_ezcase(outputs_dir, inputs_dir)
case_path = get_case_inputs_path(inputs_dir)
pairs = add_subsurfaces(case_path)
